# 游戏玩家付费金额预测大赛 - 特征工程
全部数据，分成5份。 将来再做数据平衡。

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
path_raw = '../data/raw/'
path_train = '../data/new/train=all/'
path_test = '../data/new/test/'
time_col = 'register_time'

In [3]:
train = pd.read_csv(path_raw + 'tap_fun_train.csv')
train.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,1,2018-02-02 19:47:15,20125.0,3700.0,0.0,0.0,0.0,0.0,16375.0,2000.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
1,1593,2018-01-26 00:01:05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
2,1594,2018-01-26 00:01:58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1.166667,0.0,0,0.0
3,1595,2018-01-26 00:02:13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.166667,0.0,0,0.0
4,1596,2018-01-26 00:02:46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,2.333333,0.0,0,0.0


## register_time 转换为注册天数。

In [4]:
def time_to_days(df, time_col):
    """ 玩家注册时间转换为天数 = 从注册 到抓取数据时的天数。 """
    # 报名开始：6月19日11:00   猜测抓取数据的日期 
    ty = pd.to_datetime('2018-06-11 16:51:15')
    # 采用匿名函数 lambda
    df[time_col] = df[time_col].apply(lambda x: (ty - pd.to_datetime(x)).days)
    df[time_col] = df[time_col].astype(np.int64)

In [6]:
time_to_days(train, time_col)
train.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,1,128,20125.0,3700.0,0.0,0.0,0.0,0.0,16375.0,2000.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
1,1593,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
2,1594,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1.166667,0.0,0,0.0
3,1595,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.166667,0.0,0,0.0
4,1596,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,2.333333,0.0,0,0.0


## 64位数据，转换为 32位，节省空间

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288007 entries, 0 to 2288006
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(13), int64(96)
memory usage: 1.9 GB


In [14]:
col_int64 = train.select_dtypes(include = ["int64"]).columns
col_int64.shape

(96,)

In [15]:
col_float64 = train.select_dtypes(include = ["float64"]).columns
col_float64.shape

(13,)

In [16]:
train[col_int64] = train[col_int64].astype(np.int32)
train[col_float64] = train[col_float64].astype(np.float32)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288007 entries, 0 to 2288006
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float32(13), int32(96)
memory usage: 951.4 MB


## 拆分为 5 份，随机抽样

In [17]:
# 先取出 0.2 剩余的 0.8 再平分为 0.4 再平分为 0.2 
train_part2345, train_part1 = train_test_split(train, random_state=33, test_size=0.2)
print('train.shape=%s train_part2345.shape=%s train_part1.shape=%s'
      %(train.shape, train_part2345.shape, train_part1.shape))

train.shape=(2288007, 109) train_part2345.shape=(1830405, 109) train_part1.shape=(457602, 109)


In [19]:
# train=fe-1.csv 特征工程后的，注册日期转换为天数
train_part1.to_csv(path_train + 'train=fe-1.csv', index=False)
train_part1.shape

(457602, 109)

In [21]:
train_part23, train_part45 = train_test_split(train_part2345, random_state=33, test_size=0.5)
print('train_part2345.shape=%s train_part23.shape=%s train_part45.shape=%s'
      %(train_part2345.shape, train_part23.shape, train_part45.shape))

train_part2, train_part3 = train_test_split(train_part23, random_state=33, test_size=0.5)
print('train_part23.shape=%s train_part2.shape=%s train_part3.shape=%s'
      %(train_part23.shape, train_part2.shape, train_part3.shape))
train_part2.to_csv(path_train + 'train=fe-2.csv', index=False)
train_part3.to_csv(path_train + 'train=fe-3.csv', index=False)

train_part4, train_part5 = train_test_split(train_part45, random_state=33, test_size=0.5)
print('train_part45.shape=%s train_part4.shape=%s train_part5.shape=%s'
      %(train_part45.shape, train_part4.shape, train_part5.shape))
train_part4.to_csv(path_train + 'train=fe-4.csv', index=False)
train_part4.to_csv(path_train + 'train=fe-5.csv', index=False)

train_part2345.shape=(1830405, 109) train_part23.shape=(915202, 109) train_part45.shape=(915203, 109)
train_part23.shape=(915202, 109) train_part2.shape=(457601, 109) train_part3.shape=(457601, 109)
train_part45.shape=(915203, 109) train_part4.shape=(457601, 109) train_part5.shape=(457602, 109)


## 进一步排查数据类型，float 转换为 int

In [4]:
train = pd.read_csv(path_train + 'train=fe-1.csv')
train.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,332994,112,43413.0,18424.0,0.0,0.0,0.0,0.0,62775.0,9444.0,...,0,0,0,0,0,0,0.500000,0.0,0,0.0
1,2186887,136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
2,383991,134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.666667,0.0,0,0.0
3,102315,134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.500000,0.0,0,0.0
4,225310,106,28975.0,5780.0,0.0,0.0,0.0,0.0,33525.0,3220.0,...,0,0,0,0,0,0,0.666667,0.0,0,0.0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457602 entries, 0 to 457601
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(13), int64(96)
memory usage: 380.5 MB


In [10]:
col_float64 = train.select_dtypes(include = ["float64"]).columns
col_float64

Index(['wood_add_value', 'wood_reduce_value', 'stone_add_value',
       'stone_reduce_value', 'ivory_add_value', 'ivory_reduce_value',
       'meat_add_value', 'meat_reduce_value', 'magic_add_value',
       'magic_reduce_value', 'avg_online_minutes', 'pay_price',
       'prediction_pay_price'],
      dtype='object')

In [13]:
should_int32 = ['wood_add_value', 'wood_reduce_value', 'stone_add_value',
       'stone_reduce_value', 'ivory_add_value', 'ivory_reduce_value',
       'meat_add_value', 'meat_reduce_value', 'magic_add_value',
       'magic_reduce_value']
train[should_int32] = train[should_int32].astype(np.int32)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457602 entries, 0 to 457601
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(3), int32(10), int64(96)
memory usage: 363.1 MB


In [14]:
# train=fe-1.csv 特征工程后的，注册日期转换为天数
train.to_csv(path_train + 'train=fe-1.csv', index=False)
train.shape

(457602, 109)

In [15]:
train = pd.read_csv(path_train + 'train=fe-2-1.csv')
train[should_int32] = train[should_int32].astype(np.int32)
train.to_csv(path_train + 'train=fe-2.csv', index=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457601 entries, 0 to 457600
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(3), int32(10), int64(96)
memory usage: 363.1 MB


In [17]:
train = pd.read_csv(path_train + 'train=fe-3-1.csv')
train[should_int32] = train[should_int32].astype(np.int32)
train.to_csv(path_train + 'train=fe-3.csv', index=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457601 entries, 0 to 457600
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(3), int32(10), int64(96)
memory usage: 363.1 MB


In [18]:
train = pd.read_csv(path_train + 'train=fe-4-1.csv')
train[should_int32] = train[should_int32].astype(np.int32)
train.to_csv(path_train + 'train=fe-4.csv', index=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457601 entries, 0 to 457600
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(3), int32(10), int64(96)
memory usage: 363.1 MB


In [19]:
train = pd.read_csv(path_train + 'train=fe-5-1.csv')
train[should_int32] = train[should_int32].astype(np.int32)
train.to_csv(path_train + 'train=fe-5.csv', index=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457601 entries, 0 to 457600
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(3), int32(10), int64(96)
memory usage: 363.1 MB


### 测试集

In [3]:
test = pd.read_csv(path_test + 'tap_fun_test-fe-32bit.csv')
test.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,sr_rss_help_bonus_level,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count
0,14933,94,166415,138362,0,0,0,0,258522,90142,...,0,0,0,0,1,1,1,8.000000,0.0,0
1,14934,94,10000,600,0,0,0,0,10000,400,...,0,0,0,0,0,0,0,0.166667,0.0,0
2,14935,94,10000,3700,0,0,0,0,10000,2000,...,0,0,0,0,0,0,0,17.000000,0.0,0
3,14936,94,210000,0,200000,0,200000,0,610000,0,...,0,0,0,0,0,0,0,1.666667,0.0,0
4,14937,94,11500,3700,0,0,0,0,11000,2000,...,0,0,0,0,0,0,0,0.333333,0.0,0


In [4]:
test_float64 = test.select_dtypes(include = ["float64"]).columns
test_float64

Index(['avg_online_minutes', 'pay_price'], dtype='object')

In [6]:
test_int64 = test.select_dtypes(include = ["int64"]).columns
test_int64

Index(['user_id', 'register_time', 'stone_add_value', 'stone_reduce_value',
       'ivory_add_value', 'ivory_reduce_value', 'meat_add_value',
       'meat_reduce_value', 'magic_add_value', 'magic_reduce_value',
       ...
       'sr_guest_troop_capacity_level', 'sr_march_size_level',
       'sr_rss_help_bonus_level', 'pvp_battle_count', 'pvp_lanch_count',
       'pvp_win_count', 'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'pay_count'],
      dtype='object', length=104)

In [7]:
should_int32 = ['wood_add_value', 'wood_reduce_value']
test[should_int32] = test[should_int32].astype(np.int32)
test[test_int64] = test[test_int64].astype(np.int32)
test[test_float64] = test[test_float64].astype(np.float32)
test.to_csv(path_test + 'tap_fun_test-fe.csv', index=False)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828934 entries, 0 to 828933
Columns: 108 entries, user_id to pay_count
dtypes: float32(2), int32(106)
memory usage: 341.5 MB
